In [ ]:
import torch
import torchvision

mobilenetv2 = torchvision.models.mobilenet_v2(weights="MobileNet_V2_Weights.IMAGENET1K_V1")

In [ ]:
import torchvision.transforms as transforms

transform_train = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomAffine(10, translate=(0.1, 0.1), scale=(0.9, 1.1), shear=10),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

transform_test = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ]
)

In [ ]:
train = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=transform_train
)
test = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=False, transform=transform_test
)

In [ ]:
class MobNetCustom(torch.nn.Module):
    def __init__(self, model):
        super(MobNetCustom, self).__init__()
        self.features = model.features
        self.avgpool1 = torch.nn.AvgPool2d(4)
        self.fc1 = torch.nn.Linear(1280, 128)
        self.fc2 = torch.nn.Linear(128, 10)
        self.dropout = torch.nn.Dropout(0.3)

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.dropout(x)
        x = torch.nn.functional.relu(x)
        return self.fc2(x)

In [ ]:
model = MobNetCustom(mobilenetv2)
for i in model.features.parameters():
    i.requires_grad = False

In [ ]:
train_loader = torch.utils.data.DataLoader(train, batch_size=192, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=192, shuffle=False)

In [ ]:
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

def train_loop(
    model,
    run_name,
    num_epochs=10,
    train_loader=train_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    criterion=criterion,
    device="cuda",
):
    writer = SummaryWriter(log_dir=f"runs/{run_name}")

    for epoch in range(num_epochs):
        model.train()
        total_train_loss, correct_train, total_train = 0.0, 0, 0

        train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1} [Train]", leave=True)
        for images, labels in train_loader_tqdm:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct_train += (predicted == labels).sum().item()
            total_train += labels.size(0)

        avg_train_loss = total_train_loss / len(train_loader)
        train_accuracy = 100 * correct_train / total_train
        
        print(f"Epoch {epoch+1} Train - Loss: {avg_train_loss:.4f}, Accuracy: {train_accuracy:.2f}%")
        writer.add_scalar("Loss/train", avg_train_loss, epoch)
        writer.add_scalar("Accuracy/train", train_accuracy, epoch)

        model.eval()
        total_test_loss, correct_test, total_test = 0.0, 0, 0
        
        test_loader_tqdm = tqdm(test_loader, desc=f"Epoch {epoch+1} [Test]", leave=True)
        with torch.no_grad():
            for images, labels in test_loader_tqdm:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                total_test_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct_test += (predicted == labels).sum().item()
                total_test += labels.size(0)

        avg_test_loss = total_test_loss / len(test_loader)
        test_accuracy = 100 * correct_test / total_test
        
        print(f"Epoch {epoch+1} Test - Loss: {avg_test_loss:.4f}, Accuracy: {test_accuracy:.2f}%")
        writer.add_scalar("Loss/test", avg_test_loss, epoch)
        writer.add_scalar("Accuracy/test", test_accuracy, epoch)

    writer.close()

In [ ]:
train_loop(model, "initial", num_epochs=5)

In [ ]:
def adjust_layers(model, num_layers):
    for i in model.features[:num_layers].parameters():
        i.requires_grad = True
    return model

In [ ]:
model = adjust_layers(model, 10)
train_loop(model, "final", num_epochs=10)

# layers

In [ ]:
for i in [2, 5, 10]:
    model = MobNetCustom(mobilenetv2).to("cuda")
    model = adjust_layers(model, i)
    print(i)
    train_loop(model, f"{i}-layers", num_epochs=5)

# lr

In [ ]:
for i in [0.01, 0.05, 0.001, 0.005]:
    model = MobNetCustom(mobilenetv2).to("cuda")
    optimizer = torch.optim.Adam(model.parameters(), lr=i)
    train_loop(model, f"lr-{i}", num_epochs=5, optimizer=optimizer)

# batch size

In [ ]:
for i in [32, 64, 128]:
    train_loader = torch.utils.data.DataLoader(train, batch_size=i, shuffle=True)
    test_loader = torch.utils.data.DataLoader(test, batch_size=i, shuffle=False)
    model = MobNetCustom(mobilenetv2).to("cuda")
    train_loop(model, f"batchsize-{i}", num_epochs=5, train_loader=train_loader, test_loader=test_loader)

# final

In [ ]:
model = MobNetCustom(mobilenetv2).to("cuda")
model = adjust_layers(model, 10)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
train_loader = torch.utils.data.DataLoader(train, batch_size=192, shuffle=True)
test_loader = torch.utils.data.DataLoader(test, batch_size=192, shuffle=False)
train_loop(
    model,
    "final",
    num_epochs=10,
    optimizer=optimizer,
    train_loader=train_loader,
    test_loader=test_loader,
)